In [48]:
import typing
from collections import Iterable
from enum import Enum
from pprint import pprint
from typing import List, Optional, Union, Dict
import attr
from attr import attrib, attrs
from cerberus import Validator

from mimosa.models import CraftTypes, craft_type_codes

In [49]:
def in_valid_craft_types(field, value, error) -> None:
  check = value in [craft_type_codes[e] for e in CraftTypes]
  if check is False:
    error(field, f"{value} not found in CraftTypes")


@attrs(auto_attribs=True)
class SiteKeyCompany:
    name: str
    craftTypes: List[int]
    mainPointOfContact: Optional[str]
    logoPhoto_URL: Optional[str]
    members: List[str]
    
    schema: typing.ClassVar[Dict] = {
     "name": {
       'type': 'string',
       'maxlength': 200,
     },
     "craftTypes": {
       'type': 'list',
       'schema': {
         'type': 'integer',
         'allowed': list(craft_type_codes.values())
       },
     },
     "mainPointOfContact": {
       'type': 'string',
       'maxlength': 200},
     "logoPhoto_URL": {
       'type': 'string',
       'maxlength': 1000,
       'nullable': True
     },
     "members": {
       'type': 'list', 
       'schema': {'type': 'string'}
     },
                             }


In [50]:
# site_key_company_schema = {
#   "name": {
#     'type': 'string',
#     'maxlength': 200,
#   },
#   "craftTypes": {
#     'type': 'list',
#     'schema': {
#       'type': 'integer',
#       'allowed': list(craft_type_codes.values())
#     },
#   },
#   "mainPointOfContact": {
#     'type': 'string',
#     'maxlength': 200},
#   "logoPhoto_URL": {
#     'type': 'string',
#     'maxlength': 1000,
#     'nullable': True},
#   "members": {
#     'type': 'list', 
#     'schema': {'type': 'string'}
#   },
# }

In [51]:
v = Validator(SiteKeyCompany.schema)
v.require_all = True

In [52]:
data = SiteKeyCompany(
  name="Monomer Software",
  craftTypes=[
    craft_type_codes[CraftTypes.SCAFFOLDING],
    craft_type_codes[CraftTypes.PERMITTING],
    craft_type_codes[CraftTypes.INSULATION],
    craft_type_codes[CraftTypes.HOUSEKEEPING],
    '90',
    '10'
  ],
  mainPointOfContact="Harry",
  logoPhoto_URL=None,
  members=['memberID'],
)
data_dict = attr.asdict(data)
pprint(data_dict)
is_valid = v.validate(data_dict)
pprint(is_valid)
pprint(v.errors)
valid_data = {key:value for key, value in data_dict.items() if key not in v.errors.keys()}
pprint(valid_data)

{'craftTypes': [10, 90, 15, 99, '90', '10'],
 'logoPhoto_URL': None,
 'mainPointOfContact': 'Harry',
 'members': ['memberID'],
 'name': 'Monomer Software'}
False
{'craftTypes': [{4: ['must be of integer type'],
                 5: ['must be of integer type']}]}
{'logoPhoto_URL': None,
 'mainPointOfContact': 'Harry',
 'members': ['memberID'],
 'name': 'Monomer Software'}


In [53]:
v.validate({
  'name': "Monomer",
  'mainPointOfContact': 'Daniel',
  'moose': "coffee"
})
pprint(v.errors)

{'craftTypes': ['required field'],
 'logoPhoto_URL': ['required field'],
 'members': ['required field'],
 'moose': ['unknown field']}


In [54]:
v.require_all = False
v.validate({
  'name': "Monomer",
  'mainPointOfContact': 'Daniel',
})
pprint(v.errors)


{}
